In [10]:
import torch
from torch import nn
from gensim.models import KeyedVectors

In [3]:
text = """
hier , je disais qu’ une unanimité molle régnait sur le québec .    pas moyen de sortir du rang sans que les ténors du grand bêlement collectif nous remettent à l’ ordre .    il faut penser de telle façon , agir de telle façon et mener sa barque de telle façon , sinon les gardiens de la bonne parole qui officient au tribunal du peuple nous excommunient .    « les pendules à l’ heure »   prenez le chroniqueur de télé richard therrien , du journal le soleil .   dimanche soir , sur son blogue , therrien a publié un texte sur tout le monde en parle .    « le président du syndicat des employés du journal de montréal , le photographe reynald leblanc , a remis certaines pendules à l’ heure , a -t -il écrit . contrairement à ce qu’ a affirmé pierre - karl péladeau , les employés ne gagnent pas en moyenne 88 000 $ . c’ est plutôt le salaire maximum qu’ ils peuvent atteindre . on parle plutôt d’ autour de 50 000 $ en moyenne . »   therrien n’ a pas écrit : « selon le président du syndicat du journal de montréal , les employés ne gagneraient pas 88 000 $ » , comme l’ aurait fait n’ importe quel journaliste digne de ce nom .   il a écrit : « les employés ne gagnent pas 88 000 $ . »   il a publié ça comme si c’ était un fait objectif , coulé dans le béton .    pourquoi ? parce qu’ il avait les vrais chiffres en main , parce qu’ il a effectué une longue enquête lui permettant d’ authentifier l’ affirmation de reynald leblanc ?   non . parce que c’ est le président d’ un syndicat qui l’ a dit .   si c’ est le président d’ un syndicat qui l’ a dit , ça doit donc être vrai .   et après ça , ça se dit journaliste ...    parole d’ évangile   c’ est ça , l’ unanimité molle du québec .   un homme d’ affaires lance une affirmation — on utilise les guillemets et le futur antérieur afin d’ indiquer que c’ est sa vision des choses , et qu’ il faut prendre ses propos avec un gros grain de sel .   un chef syndical lance une affirmation — on écrit ça au présent et sans guillemets , comme si c’ était un fait indéniable .    l’ un est présumé coupable jusqu’ à preuve du contraire . l’ autre a notre bénédiction .    l' un a le fardeau de la preuve . l' autre a le bénéfice du doute .   pourquoi ce double standard ?   un politicien ou un entrepreneur ouvre la bouche , on se dit : « il est probablement en train de nous crosser . »   le porte-parole d’ un groupe de pression ou d’ un syndicat ouvre la bouche , on se dit : « cet homme dit la vérité . »   comme si les gens à gauche ne mentaient pas ! comme s’ ils ne tordaient pas les chiffres ! comme s’ ils étaient ontologiquement , essentiellement , fondamentalement bons !   comme si tout ce qui sortait de leur bouche était parole d’ évangile !    journalisme amateur   ce que richard therrien a fait est indigne d’ un journal comme la presse ou le soleil . pourtant , ça passe . c’ est publié . c’ est présenté comme un fait objectif .    mais essayez l’ inverse , vous . essayez de faire passer les dires d’ un entrepreneur ou d’ un politicien comme la vérité , sans aucune forme de vérification . on va vous tomber dessus en disant — avec raison — que vous faites un travail bâclé .   c’ est ça , le québec .   la droite est toujours coupable . la gauche est toujours innocente .    les patrons sont toujours menteurs . les syndicats sont toujours honnêtes .    les hommes sont toujours bourreaux . les femmes sont toujours victimes .    la cause est toujours entendue avant même que les faits ne soient présentés et que le procès commence .
"""

In [4]:
vocab = set(text.split())
word_to_idx = {word: i for i, word in enumerate(sorted(vocab))} 

In [5]:
len(word_to_idx)

265

In [8]:
word_to_idx

{'!': 0,
 '$': 1,
 ',': 2,
 '-': 3,
 '-il': 4,
 '-t': 5,
 '.': 6,
 '...': 7,
 '000': 8,
 '50': 9,
 '88': 10,
 ':': 11,
 '?': 12,
 'a': 13,
 'affaires': 14,
 'affirmation': 15,
 'affirmé': 16,
 'afin': 17,
 'agir': 18,
 'amateur': 19,
 'antérieur': 20,
 'après': 21,
 'atteindre': 22,
 'au': 23,
 'aucune': 24,
 'aurait': 25,
 'authentifier': 26,
 'autour': 27,
 'autre': 28,
 'avait': 29,
 'avant': 30,
 'avec': 31,
 'barque': 32,
 'blogue': 33,
 'bonne': 34,
 'bons': 35,
 'bouche': 36,
 'bourreaux': 37,
 'bâclé': 38,
 'bénédiction': 39,
 'bénéfice': 40,
 'béton': 41,
 'bêlement': 42,
 'cause': 43,
 'ce': 44,
 'certaines': 45,
 'cet': 46,
 'chef': 47,
 'chiffres': 48,
 'choses': 49,
 'chroniqueur': 50,
 'collectif': 51,
 'comme': 52,
 'commence': 53,
 'contraire': 54,
 'contrairement': 55,
 'coulé': 56,
 'coupable': 57,
 'crosser': 58,
 'c’': 59,
 'dans': 60,
 'de': 61,
 'des': 62,
 'dessus': 63,
 'digne': 64,
 'dimanche': 65,
 'dires': 66,
 'disais': 67,
 'disant': 68,
 'dit': 69,
 'doit'

In [6]:
# Loading previously created word embeddings using gensim
vec_model_path = '/home/ngarneau/workspace/fastText/result/martibot-non-subword.vec'
vec_model = KeyedVectors.load_word2vec_format(vec_model_path)

In [7]:
len(vec_model.vocab)

85832

In [11]:
class MyEmbeddings(nn.Embedding):
    def __init__(self, word_to_idx, embedding_dim):
        super().__init__(len(word_to_idx), embedding_dim, padding_idx=0)
        self.embedding_dim = embedding_dim
        self.vocab_size = len(word_to_idx)
        self.word_to_idx = word_to_idx

    def set_item_embedding(self, idx, embedding):
        self.weight.data[idx] = torch.FloatTensor(embedding)

    def load_words_embeddings(self, vec_model):
        for word in vec_model.index2word:
            if word in self.word_to_idx:
                idx = self.word_to_idx[word]
                embedding = vec_model[word]
                self.set_item_embedding(idx, embedding)
                
embeddings_layer = MyEmbeddings(word_to_idx, vec_model.vector_size)
embeddings_layer.load_words_embeddings(vec_model)

In [14]:
embeddings_layer(torch.LongTensor([word_to_idx['fardeau']]))

tensor([[-0.5302,  1.5655,  0.1522, -2.8967, -1.2558,  0.2743, -0.0139, -1.4988,
          0.4073,  1.4930, -1.6155, -1.3601,  1.9467,  2.4351,  1.5251,  1.2024,
         -1.5219, -0.7720,  2.8295, -1.1596, -1.8699,  1.9901, -1.2722,  0.4328,
          0.2930,  1.9308, -0.9047,  0.9654,  0.7150, -2.0367, -0.1484,  1.4563,
         -2.0209,  0.1235,  1.8981,  0.7131, -1.4859, -2.4792, -1.1683, -1.0590,
         -2.1819, -2.0112, -3.5989, -1.5333, -0.0342, -1.3583, -1.1768, -2.1331,
          0.8940,  1.7897, -1.9837,  0.7402, -0.3842,  2.2458,  2.1906,  0.5673,
          0.8822,  1.6477, -2.0023, -0.7556, -4.4288,  2.9440,  1.6703, -0.0390,
          2.5401, -0.0503, -1.2785, -1.1600, -1.0253,  0.3945, -1.1518, -1.9466,
          2.4659,  1.8640,  2.4385,  0.5507, -1.2993,  2.9220,  0.2024, -0.0215,
          2.1033, -3.2927, -2.5581,  0.0662,  0.8940, -0.1472,  1.4989,  0.4765,
         -1.8072,  0.7090, -2.7256,  1.1555, -0.7732, -0.6978,  2.3139,  0.1457,
         -1.3956,  1.4709,  

In [19]:
vec_model['fardeau'] == embeddings_layer(torch.LongTensor([word_to_idx['fardeau']])).detach().numpy()

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True]], dtype=bool)